In [2]:
# Install the requirements:
!pip install streamlit transformers torch sentencepiece pyngrok

In [3]:
# Create the app.py:
%%writefile app.py
import streamlit as st
from transformers import pipeline

# Set the title and a brief description
st.title("Text Summarization App")
st.write("Enter some text and I'll generate a summary for you!")

# Load the summarization pipeline
@st.cache_resource
def load_summarizer():
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    return summarizer

summarizer = load_summarizer()

# Text input area
text_input = st.text_area("Paste your text here:", height=200)

# Summarization button
if st.button("Generate Summary"):
    if text_input:
        # Generate summary
        summary = summarizer(text_input, max_length=130, min_length=30, do_sample=False)

        # Display the summary
        st.subheader("Summary:")
        st.write(summary[0]['summary_text'])
    else:
        st.warning("Please enter some text to summarize.")

Writing app.py


In [4]:
# Run the actual app:
import threading
import subprocess
import time
import re
from pyngrok import ngrok, conf
import os
import sys

# Set your ngrok auth token here (optional but recommended)
# Note: Unless you want to modify this code, you must get an auth token and put it below. Sign up here: https://ngrok.com/
# Then go to your dashboard to get the authentication token.
NGROK_AUTH_TOKEN = "YOUR TOKEN HERE" #os.environ.get("NGROK_AUTH_TOKEN", None)
if NGROK_AUTH_TOKEN:
    conf.get_default().auth_token = NGROK_AUTH_TOKEN

# Function to run Streamlit
def run_streamlit():
    try:
        process = subprocess.Popen(
            ["streamlit", "run", "app.py", "--server.port", "8502", "--server.address", "127.0.0.1"],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True, # Ensure text output for proper decoding
        )
        # Capture and print both stdout and stderr
        for line in process.stdout:
            print(f"STREAMLIT STDOUT: {line.strip()}")
        for line in process.stderr:
            print(f"STREAMLIT STDERR: {line.strip()}")

    except Exception as e:
        print(f"Error running Streamlit: {e}")

# Function to run ngrok
def run_ngrok():
    try:
        time.sleep(10)  # Increased delay to allow Streamlit to fully start
        public_url = ngrok.connect(8502)
        print("Public URL:", public_url)

        # Extract and print the ngrok URL using regex
        match = re.search(r"\"(https://.*\.ngrok-free\.app)\"", str(public_url))
        if match:
            print("Ngrok URL:", match.group(1))
        else:
            print("Failed to extract ngrok URL. Check the ngrok output for details.")

    except Exception as e:
        print(f"Error running ngrok: {e}")

# Create and start threads
streamlit_thread = threading.Thread(target=run_streamlit)
ngrok_thread = threading.Thread(target=run_ngrok)

streamlit_thread.start()
ngrok_thread.start()

# Keep the main thread alive
while True:
    time.sleep(60)

STREAMLIT STDOUT: 
STREAMLIT STDOUT: Collecting usage statistics. To deactivate, set browser.gatherUsageStats to false.
STREAMLIT STDOUT: 
STREAMLIT STDOUT: 
STREAMLIT STDOUT: You can now view your Streamlit app in your browser.
STREAMLIT STDOUT: 
STREAMLIT STDOUT: URL: http://127.0.0.1:8502
STREAMLIT STDOUT: 
Public URL: NgrokTunnel: "https://c4c8-34-126-107-14.ngrok-free.app" -> "http://localhost:8502"
Ngrok URL: https://c4c8-34-126-107-14.ngrok-free.app
STREAMLIT STDOUT: Stopping...


KeyboardInterrupt: 